In [ ]:
import numpy as np

def NAG_MBGD_LR(x, y, alpha=0.05, batch_size=100, momentam_gama=0, epochs=10, gradient_norm=0.001, loss_condition=0.001):
    # --- Data Preprocessing Phase ---
    m = x.shape[0]
    x = np.array(x)
    y = np.array(y).reshape(-1, 1)
    
    # Shuffle data to ensure mini-batches are representative of the distribution
    all_data = np.concatenate((x, y), axis=1)
    np.random.shuffle(all_data)
    
    # Split back into features and target, adding the bias term (intercept)
    x = all_data[:, :-1]
    x = np.concatenate((np.ones((m, 1)), x), axis=1)
    y = all_data[:, -1].reshape(-1, 1)
    
    # --- Initialization Phase ---
    m = x.shape[0]
    loss = []
    w = np.zeros(x.shape[1]).reshape(-1, 1)      # Model weights
    thetas = []                                  # History of weights for tracking
    total_loss = []                              # Loss accumulated per epoch
    momentam = np.zeros(x.shape[1]).reshape(-1, 1) # Velocity/Momentum vector
    
    # --- Training Loop ---
    for i in range(epochs):
        start = 0 
        end = 0
        total_epoch_loss = 0
        
        # Iterating through data in mini-batches
        while(start < m):
            end = start + batch_size if (start + batch_size) < m else m
            x_temp = x[start:end, :]
            y_temp = y[start:end, :]
            m_temp = x_temp.shape[0]
            
            # NAG Step 1: Look ahead by moving weights according to current momentum
            w_temp = w - momentam_gama * momentam
            
            # Forward pass: Prediction and Error calculation
            hx = x_temp @ w_temp
            e = hx - y_temp
            
            # Calculate Mean Squared Error for the batch
            j = e.T @ e / (2 * m_temp)
            loss.append(j)
            
            # NAG Step 2: Compute gradient at the "looked-ahead" position
            G = (x_temp.T @ e) / m_temp
            
            # NAG Step 3: Update velocity and weights
            vt = alpha * G + momentam_gama * momentam
            thetas.append(w.copy())
            start = end
            w = w - vt
            momentam = vt # Update momentum for the next step
            
            total_epoch_loss += np.absolute(j)
            
        total_loss.append(total_epoch_loss)
        
        # --- Early Stopping Criteria ---
        # 1. Stop if the gradient norm is sufficiently small
        if np.linalg.norm(G) <= (gradient_norm):
             print(f'Gradient Norm condition met. Stopped at epoch {i}')
             break
             
        # 2. Stop if the change in total loss between epochs is negligible
        if i > 2 and np.absolute(total_loss[i-1] - total_loss[i]) < loss_condition:
            print(f'Loss convergence condition met. Stopped at epoch {i}')
            break
            
    # --- Evaluation Phase (R2 Score calculation) ---
    y_mean = np.mean(y)
    ss_res, ss_tot = 0, 0
    for s in range(0, m, batch_size):
        e = min(s + batch_size, m)
        y_p = x[s:e, :] @ w
        ss_res += np.sum((y[s:e] - y_p)**2)
        ss_tot += np.sum((y[s:e] - y_mean)**2)
    
    print(f'Final R2 Score = {1 - (ss_res / ss_tot)}') 
    print('Execution completed.')
    
    return np.array(w), np.array(loss), np.array(thetas)

In [ ]:
def NAG_MBGD_LR(x,y,  alpha=0.05,batch_size=100 , momentam_gama=0,epochs=10 ,  gradient_norm=0.001  ,  loss_condition=0.001):
    m=x.shape[0]
    x=np.array(x)
    y=np.array(y).reshape(-1,1)
    all_data=np.concatenate((x,y),axis=1)
    np.random.shuffle(all_data)
    x=all_data[:,:-1]
    x = np.concatenate((np.ones((m, 1)), x), axis=1)
    y=all_data[:,-1].reshape(-1,1)
    m=x.shape[0]
    loss=[]
    w=np.zeros(x.shape[1]).reshape(-1,1)
    w_temp=np.zeros(x.shape[1]).reshape(-1,1)
    thetas=[] # return tensor 
    total_loss=[]
    momentam=np.zeros(x.shape[1]).reshape(-1,1)
    for i in range (epochs) :
        start=0 
        end= 0
        total_epoch_loss=0
        
        while(start<m):
            end=start+batch_size if (start+batch_size)<m else m
            x_temp=x[start:end,:]
            y_temp=y[start:end,:]
            m_temp=x_temp.shape[0]
            w_temp=w-momentam_gama*momentam
            hx=x_temp@w_temp
            e=hx-y_temp
            j=e.T@e/(2*m_temp)
            loss.append(j)
            G=(x_temp.T@e)/m_temp
            vt=alpha*G+momentam_gama*momentam
            thetas.append(w.copy())
            start=end
            w=w-vt
            momentam=vt
            total_epoch_loss+=np.absolute(j)
        total_loss.append(total_epoch_loss)
        if np.linalg.norm(G)<=(gradient_norm):
             print (f''' Gradient Norm condation
             stop in epoch number {i} 
             gredient norm = {np.linalg.norm(G)}''')
             break
        if i>2 and np.absolute(total_loss[i-1]-total_loss[i])<loss_condition:
            print (f'''loss condation
            stop in epoch number {i} 
            gredient norm = {np.linalg.norm(G)}
            loss[i]-loss[i-1]={total_loss[i-1]-total_loss[i]}
            ''')
            break
    y_mean = np.mean(y)
    ss_res, ss_tot = 0, 0
    for s in range(0, m, batch_size):
        e = min(s + batch_size, m)
        y_p = x[s:e, :] @ w
        ss_res += np.sum((y[s:e] - y_p)**2)
        ss_tot += np.sum((y[s:e] - y_mean)**2)
    
    print(f'Final R2 Score = {1 - (ss_res / ss_tot)}') 
    print('you finshed your itirations ')
    
    return np.array (w) ,np.array(loss),np.array(thetas)
            
    